In [1]:
import datetime as dt
from pathlib import Path

import pandas as pd
import numpy as np
from highcharts import Highchart

DIR = Path('..')

In [6]:
# Load data
paths = [
    'transactions_fy_2014--2015.csv',
    'transactions_fy_2015--2016.csv',
    ]
paths = [DIR/'tumeke_cycle_space'/'finances'/p for p in paths]
parser = lambda date: pd.datetime.strptime(date, '%d-%m-%Y')
frames = [pd.read_csv(p, parse_dates=['Date'], date_parser=parser)
  for p in paths]
f = pd.concat(frames)

# Rename columns for convenience
columns1 = {
  c: c.lower().strip().replace(' ', '_')
  for c in f.columns
  }
columns2 = {
  'memo/description': 'memo',
  'amount_(credit)': 'credit',
  'amount_(debit)': 'debit',
  }
f = f.rename(columns=columns1).rename(columns=columns2)
f.head().T


NameError: name 'DIR' is not defined

In [3]:
g = f[['date', 'credit', 'debit', 'balance']].copy()
g = g.set_index('date')

# Resample to month
g = g.resample('M').sum().fillna(0)

# Create cumulative moving averages
for col in g.columns:
    new_col = col + '_cma'
    g[new_col] = g[col].expanding(min_periods=1).mean() 

g

,credit,debit,balance,credit_cma,debit_cma,balance_cma
date,,,,,,
2014-07-31,155.50,541.50,10509.26,155.500000,541.500000,10509.260000
2014-08-31,124.50,320.00,8001.55,140.000000,430.750000,9255.405000
2014-09-30,246.90,320.00,4912.74,175.633333,393.833333,7807.850000
2014-10-31,2500.00,320.00,8416.33,756.725000,375.375000,7959.970000
2014-11-30,160.00,320.00,13418.44,637.380000,364.300000,9051.664000
2014-12-31,800.00,480.00,17560.55,664.483333,383.583333,10469.811667
2015-01-31,0.00,538.20,16480.95,569.557143,405.671429,11328.545714
2015-02-28,1070.00,340.96,17304.75,632.112500,397.582500,12075.571250
2015-03-31,0.00,320.00,7285.90,561.877778,388.962222,11543.385556


In [4]:
# Plot with Highcharts

chart = Highchart()
options = {
    'chart': {
        'width': 700,
    },
    'title': {
        'text': 'Tumeke Cycle Space Finances'
    },
    'xAxis': {
        'type': 'datetime',
        'labels': {
            'format': '{value:%Y-%m}',
            'rotation': -45,
            'align': 'right',
        },
        'tickInterval': 28*24*3600*1000,
    },
    'yAxis': {
        'title': {
            'text': ''
        },
        'labels': {
            'formatter': "function () {\
                return '$' + this.value;\
            }"
        },
        'lineWidth': 2
    },
    'tooltip': {
        'headerFormat': '<b>{series.name}</b><br/>',
        'pointFormat': '{point.x: %Y-%m}, ${point.y:.0f}',
    }
}
chart.set_dict_options(options)

chart.add_data_set(g[['credit']].reset_index().values.tolist(), 
  'column', 'Credit') 
chart.add_data_set(g[['credit_cma']].reset_index().values.tolist(), 
  'spline', 'Credit Cumulative Moving Average') 
chart.add_data_set(g[['debit']].reset_index().values.tolist(),
  'column', 'Debit')
chart.add_data_set(g[['debit_cma']].reset_index().values.tolist(), 
   'spline', 'Debit Cumulative Moving Average') 

chart

In [5]:
chart.add_data_set?